In [2]:
import torch
from torch.autograd import Variable
import math
import pyro
import pyro.distributions as dist

In [3]:
print pyro.__version__

0.1.2


In [4]:
mu = Variable(torch.zeros(1))
sigma = Variable(torch.ones(1))
x = dist.normal(mu,sigma)
print(x)

Variable containing:
-1.8659
[torch.FloatTensor of size 1]



In [5]:
log_p_x = dist.normal.log_pdf(x,mu,sigma)
print(log_p_x)
print(math.exp(log_p_x.data[0]))

Variable containing:
-2.6597
[torch.FloatTensor of size 1]

0.0699716507865


In [6]:
x = pyro.sample("test_sampler", dist.normal,mu,sigma)
print(x)
#can also define your own arbitrary stochastic function and do
# x = pyro.sample("my_sample", fn, arg1, arg2)

Variable containing:
 2.2550
[torch.FloatTensor of size 1]



In [7]:
def weather():
    cloudy = pyro.sample('cloudy',dist.bernoulli,
                        Variable(torch.Tensor([0.3])))
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny'
    mean_temp = {'cloudy':[55.0] , 'sunny':[75.0]}[cloudy]
    sigma_temp = {'cloudy':[10.0] , 'sunny':[15.0]}[cloudy]
    temp = pyro.sample('temp', dist.normal, Variable(torch.Tensor(mean_temp)), Variable(torch.Tensor(sigma_temp)))
    return cloudy,temp.data[0]

In [8]:
for _ in range(3):
    print (weather())

('sunny', 76.04932403564453)
('sunny', 64.96636962890625)
('sunny', 101.13807678222656)


# now lets build a larger model from the simpler model

In [12]:
def ice_cream_sales():
    cloudy,temp = weather()
    expected_sales = [200] if cloudy == 'sunny' and temp > 80.0 else [50]
    ice_cream = pyro.sample('ice_cream', dist.normal, 
                            Variable(torch.Tensor(expected_sales)),Variable(torch.Tensor(sigma_temp)))
    return ice_cream

# more complex flows: Recursion, Higher order stochastic, Random Control Flow

In [13]:
def geometric(p, t= None):
    if t is None:
        t = 0
    #in the line below, the format function updates the sample name based on t
    x = pyro.sample("x_{}".format(t), dist.bernoulli,p)
    if torch.equal(x.data, torch.zeros(1)): #hmmm..annoying tensor comparisons
        return x
    else:
        return x + geometric(p, t+1) # stochastic recursion. So if 3 is returned then we "recursed" :-) in thrice

In [14]:
print (geometric(Variable(torch.Tensor([0.5]))))

Variable containing:
 1
[torch.FloatTensor of size 1]



In [16]:
def normal_product (mu,sigma):
    z1 = pyro.sample(("z1"), dist.normal, mu, sigma)
    z2 = pyro.sample(("z2"), dist.normal, mu, sigma)
    y = z1*z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample('mu_latent', dist.normal, Variable(torch.zeros(1)),Variable(torch.ones(1)))
    fn = lambda sigma: normal_product(mu_latent,sigma)
    return fn #so at this point we return a FUNCTION that upon using with an argument sigma, give 
              # a value from normal_product(mu_latent,sigma)
    

In [19]:
print(make_normal_normal()(Variable(torch.ones(1))))

Variable containing:
 3.0580
[torch.FloatTensor of size 1]

